First imports

In [0]:
from google.colab import files
import torch
from torch.utils.data.dataset import Dataset
import numpy as np
from google.colab import drive
import torch.nn as nn
drive.mount('/content/drive')
import random

import time
device = torch.device("cuda")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Let's now load the data


In [0]:
class Data(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []
        self.load_data()
        self.N_TOKENS = len(self.idx2word)

    def load_data(self):
        lines = 0
        self.idx2word.append('<E>')
        self.word2idx['<E>'] = len(self.idx2word)-1
        tokens = 0
        with open('/content/drive/My Drive/shortjokes_noquote.txt', 'r') as data:
          for line in data:
              words = line.split() + ['<EOS>']
              lines = lines + 1
              tokens = tokens + len(words)
              for word in words:
                  if word not in self.word2idx:
                      self.idx2word.append(word)
                      self.word2idx[word] = len(self.idx2word)-1

        self.inputs = []
        self.ids = torch.LongTensor(tokens)
        token = 0
        with open('/content/drive/My Drive/shortjokes_noquote.txt', 'r') as data:
          for line in data:
              words = line.split() + ['<EOS>']
              ind = 0
              input = torch.zeros([len(words)], dtype=torch.int64)
              for word in words:
                  input[ind] = self.word2idx[word]
                  self.ids[token] = self.word2idx[word]
                  token = token+1
                  ind = ind+1
              self.inputs.append(input)
        data.close()
        print("ids ", len(self.ids), " tokens ", len(self.idx2word))
        print(len(input), " tensors.")

class JokeData(Dataset):
    def __init__(self):
        self.data = Data()
        self.EOS_token = self.data.word2idx['<EOS>']

    def __len__(self):
        return len(self.data.inputs)

    def __getitem__(self, idx):
        sample = self.data.inputs[idx]
        return sample

Let's now define the model

In [0]:
class WordRNN(nn.Module):
    def __init__(self, ntoken, ninp=200, nhid = 128, nlayers = 1, dropout=0.2):
        super(WordRNN, self).__init__()
        self.embedding = nn.Embedding(ntoken, ninp)
        self.lstm = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)

        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        emb = self.embedding(input)
        output, hidden = self.lstm(emb, hidden)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                weight.new_zeros(self.nlayers, bsz, self.nhid))

In [0]:
def batchify(data, bsz):
    nbatch = data.size(0) // bsz
    data = data.narrow(0, 0, nbatch * bsz)
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

def get_batch(source, i):
    seq_len_tmp = min(seq_len, len(source) - 1 - i)
    data = source[i:i+seq_len_tmp]
    target = source[i+1:i+1+seq_len_tmp].view(-1)
    return data, target

trainset = JokeData()
batch_size = 64
train_data = batchify(trainset.data.ids, batch_size)
seq_len = 42


  

ids  4298259  tokens  228942
20  tensors.


In [0]:
model = WordRNN(ntoken=len(trainset.data.idx2word)).to(device)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [0]:
model.load_state_dict(torch.load("/content/drive/My Drive/lol.weight"))
model.lstm.flatten_parameters()

Training loop


In [0]:
def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)
criterion = nn.CrossEntropyLoss()
epochs = 12
lr = 20
learning_rate = 1e-4
while epochs > 0:
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    print("aloitetaan epoch")
    epochs = epochs-1
    model.train()
    total_loss = 0.
    start_time = time.time()
    ntokens = len(trainset.data.idx2word)
    hidden = model.init_hidden(batch_size)
    seq_len = 35+random.randint(-10, 10)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, seq_len)):
        data, targets = get_batch(train_data, i)
        #print(data, " data", data.shape)
        #print(targets, "targets ", targets.shape)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = repackage_hidden(hidden)
        model.zero_grad()
        optimizer.zero_grad()
        output, hidden = model(data, hidden)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        #for p in model.parameters():
        #    p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()
        optimizer.step()
        if batch % 100 == 0 and batch > 0:
            cur_loss = total_loss / 100
            elapsed = time.time() - start_time
            print("cur loss ", cur_loss)
            total_loss = 0
            start_time = time.time()

aloitetaan epoch


RuntimeError: ignored

In [0]:
torch.save(model.state_dict(), "/content/drive/My Drive/lollower.weight")


with open("malli2.lol", 'wb') as f:
                torch.save(model, f)

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type WordRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Normal sampling and top-k sampling


In [0]:
words = 1000
temp = 1.0
model.eval()
hidden = model.init_hidden(1)
input = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device)
with torch.no_grad():  # no tracking history
    for i in range(words):
        output, hidden = model(input, hidden)
        word_weights = output.squeeze().div(temp).exp().cpu()
        word_idx = torch.multinomial(word_weights, 1)[0]
        input.fill_(word_idx)
        word = trainset.data.idx2word[word_idx]

        print(word,' ', end = ('\n' if word == "<EOS>" else ' '))


to   show   me   this   Trump   jokes.   <EOS>  
I   think   I   have   the   right   words   in   your   future.   Is   it   not   child   edited:   Damn   enough   to   delete   it   or   maybe   this   earth   looks   like   cake   What   don't   you   get?   <EOS>  
A   female   frog   falls   by   a   trash   attack   on   a   two   cats   eating   apples.   <EOS>  
The   forgot   I   bad   women.   Her   name   is   flipped   of   kissing   84   Really   drunk   and   the   next   door   and   stop   taking   a   nap.   <EOS>  
So   a   day...   quietly   trying   to   make   you   PC   from   before   I   and   the   other   hand,   is.   <EOS>  
At   my   Facebook   start   the   desk   &   I   have   to   cut   the   toilet   out   of   me   and   a   cheap   it   <EOS>  
Everyone   is   going   to   meet   someone   with   really   good   shit,   but   they   don't   call   it   the   soooo   better.   <EOS>  
When   I   am   happy   to   avoid   watching   just   food   soun

In [0]:
words = 1000
temp = 1.0
model.eval()
hidden = model.init_hidden(1)
topk = 40
input = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device)
with torch.no_grad():  # no tracking history
    for i in range(words):
        output, hidden = model(input, hidden)
        weights, ind = torch.topk(output.squeeze(), topk)
        word_weights = weights.div(temp).exp().cpu()
        word_idx = ind[torch.multinomial(word_weights, 1)[0]]
        input.fill_(word_idx)
        word = trainset.data.idx2word[word_idx]

        print(word,' ', end = ('\n' if word == "<EOS>" else ' '))